In [1]:
import json
import ray
from ray.rllib.agents.registry import get_agent_class
from ray.tune import run_experiments
from ray.tune.registry import register_env
import numpy as np
from flow.networks.ring import RingNetwork, ADDITIONAL_NET_PARAMS
from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder
from flow.core.params import SumoParams, EnvParams, InitialConfig, NetParams
from flow.core.params import VehicleParams, SumoCarFollowingParams
from flow.controllers import RLController, IDMController, ContinuousRouter
from gym.spaces.box import Box
from gym.spaces import Tuple
from flow.core.params import InFlows
from flow.controllers import SimLaneChangeController
from flow.networks import Network
import os
from flow.controllers.routing_controllers import ConstructionRouter
from flow.core.params import SumoLaneChangeParams

from env_constructionV4_padding import myEnv

ADDITIONAL_ENV_PARAMS = {
    "max_accel": 1,
    "max_decel": 1,
}

# time horizon of a single rollout
HORIZON = 1500
# number of rollouts per training iteration
N_ROLLOUTS = 20
# number of parallel workers
N_CPUS = 2

# We place one autonomous vehicle and 22 human-driven vehicles in the network
vehicles = VehicleParams()
vehicles.add("rl",
             acceleration_controller=(RLController, {}),
             lane_change_controller=(SimLaneChangeController, {}),
             routing_controller=(ConstructionRouter, {}),
             car_following_params=SumoCarFollowingParams(
                 speed_mode="obey_safe_speed",  
                 # we use the speed mode "obey_safe_speed" for better dynamics at the merge
             ),
             num_vehicles=2)
vehicles.add("human",
             acceleration_controller=(IDMController, {}),
             lane_change_controller=(SimLaneChangeController, {}),
             #routing_controller=(ContinuousRouter, {}),
             car_following_params=SumoCarFollowingParams(
                 speed_mode="obey_safe_speed",  
                 # we use the speed mode "obey_safe_speed" for better dynamics at the merge
             ),
             lane_change_params = SumoLaneChangeParams(lane_change_mode="strategic",lcpushy=1.0),
             num_vehicles=0)

# specify the edges vehicles can originate on
initial_config = InitialConfig(
    edges_distribution=["gneE4","gneE35","gneE38"]
)

# specify the routes for vehicles in the network
class Network(Network):

    def specify_routes(self, net_params):
        return {
                "gneE35": ["gneE35","gneE15","gneE13","gneE4.264","gneE4.264.110","gneE8","gneE9","gneE9.252","gneE33"],
                "gneE4": ["gneE4","gneE17","gneE11","gneE13","gneE4.264","gneE4.264.110","gneE8","gneE9","gneE9.252","gneE33"],
               "gneE8": ["gneE8","gneE9","gneE37","gneE38","gneE39","gneE4.264.110","gneE8"]
               }


inflow = InFlows()
inflow.add(veh_type="human",
           edge="gneE4",
           vehs_per_hour=1000,
            depart_lane="random",
            depart_speed="random",
            color="white")
inflow.add(veh_type="human",
           edge="gneE35",
           vehs_per_hour=5000,
            depart_lane="random",
            depart_speed="random",
            color="blue")


file_dir = "/home/llavezzo/"
net_params = NetParams(
    template="/mnt/c/Users/llave/Documents/GitHub/flow_osuphysics/lucalavezzo/constructionV4.net.xml",
    inflows=inflow
)

flow_params = dict(
    # name of the experiment
    exp_tag="construction_traffic",

    # name of the flow environment the experiment is running on
    env_name=myEnv,  # <------ here we replace the environment with our new environment

    # name of the network class the experiment is running on
    network=Network,

    # simulator that is used by the experiment
    simulator='traci',

    # sumo-related parameters (see flow.core.params.SumoParams)
    sim=SumoParams(
        sim_step=0.1,
        render=False,
        restart_instance=True,
    ),

    # environment related parameters (see flow.core.params.EnvParams)
    env=EnvParams(
        horizon=HORIZON,
        warmup_steps=750,
        clip_actions=False,
        additional_params={
            "target_velocity": 20,
            "sort_vehicles": False,
            "max_accel": 1,
            "max_decel": 1,
        },
    ),

    # network-related parameters (see flow.core.params.NetParams and the
    # network's documentation or ADDITIONAL_NET_PARAMS component)
    net=net_params,

    # vehicles to be placed in the network at the start of a rollout (see
    # flow.core.params.VehicleParams)
    veh=vehicles,

    # parameters specifying the positioning of vehicles upon initialization/
    # reset (see flow.core.params.InitialConfig)
    initial=initial_config
)


def setup_exps():
    """Return the relevant components of an RLlib experiment.

    Returns
    -------
    str
        name of the training algorithm
    str
        name of the gym environment to be trained
    dict
        training configuration parameters
    """
    alg_run = "PPO"

    agent_cls = get_agent_class(alg_run)
    config = agent_cls._default_config.copy()
    config["num_workers"] = N_CPUS
    config["train_batch_size"] = HORIZON * N_ROLLOUTS
    config["gamma"] = 0.999  # discount rate
    config["model"].update({"fcnet_hiddens": [3, 3]})
    config["use_gae"] = True
    config["lambda"] = 0.97
    config["kl_target"] = 0.02
    config["num_sgd_iter"] = 10
    config['clip_actions'] = False  # FIXME(ev) temporary ray bug
    config["horizon"] = HORIZON

    # save the flow params for replay
    flow_json = json.dumps(
        flow_params, cls=FlowParamsEncoder, sort_keys=True, indent=4)
    config['env_config']['flow_params'] = flow_json
    config['env_config']['run'] = alg_run

    create_env, gym_name = make_create_env(params=flow_params, version=0)

    # Register as rllib env
    register_env(gym_name, create_env)
    return alg_run, gym_name, config




/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.

In [ ]:
alg_run, gym_name, config = setup_exps()
ray.init(num_cpus=N_CPUS + 1)
trials = run_experiments({
    flow_params["exp_tag"]: {
        "run": alg_run,
        "env": gym_name,
        "config": {
            **config
        },
        "checkpoint_freq": 20,
        "checkpoint_at_end": True,
        "max_failures": 999,
        "stop": {
            "training_iteration": 200,
        },
    }
})


2020-03-20 14:26:08,061	WARNING services.py:592 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2020-03-20 14:26:08,064	INFO resource_spec.py:212 -- Starting Ray with 4.15 GiB memory available for workers and up to 2.09 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-03-20 14:26:08,453	WARNING services.py:1080 -- Failed to start the dashboard. The dashboard requires Python 3 as well as 'pip install aiohttp psutil setproctitle grpcio'.


Trial name,status,loc
PPO_myEnv-v0_45206402,RUNNING,


(pid=2087) E0320 14:26:10.294395500    2087 socket_utils_common_posix.cc:208] check for SO_REUSEPORT: {"created":"@1584728770.294380800","description":"Protocol not available","errno":92,"file":"external/com_github_grpc_grpc/src/core/lib/iomgr/socket_utils_common_posix.cc","file_line":185,"os_error":"Protocol not available","syscall":"getsockopt(SO_REUSEPORT)"}
(pid=2087) E0320 14:26:10.294617900    2087 socket_utils_common_posix.cc:313] setsockopt(TCP_USER_TIMEOUT) Protocol not available
(pid=raylet) E0320 14:26:08.636586700    2065 socket_utils_common_posix.cc:208] check for SO_REUSEPORT: {"created":"@1584728768.636561800","description":"Protocol not available","errno":92,"file":"external/com_github_grpc_grpc/src/core/lib/iomgr/socket_utils_common_posix.cc","file_line":185,"os_error":"Protocol not available","syscall":"getsockopt(SO_REUSEPORT)"}
(pid=raylet) E0320 14:26:08.636969400    2065 socket_utils_common_posix.cc:313] setsockopt(TCP_USER_TIMEOUT) Protocol not available
(pid=ray

(pid=2086) /home/llavezzo/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=2086)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=2086) /home/llavezzo/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=2086)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=2086) /home/llavezzo/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=2086)   _np_qint16 = np.dtype([("qint16", np.int16, 1)]

(pid=2086) Warning: Vehicle 'flow_10.27' performs emergency braking with decel=-59.33 wished=4.50 severity=12.19, time=110.40.
(pid=2088) Warning: Vehicle 'flow_00.19' performs emergency braking with decel=-17.22 wished=4.50 severity=2.83, time=111.40.
(pid=2086) Warning: Vehicle 'flow_10.27' performs emergency braking with decel=-22.50 wished=4.50 severity=4.00, time=112.40.
(pid=2088) Warning: Vehicle 'flow_10.32' performs emergency braking with decel=-93.97 wished=4.50 severity=19.88, time=118.60.
(pid=2088) Warning: Vehicle 'flow_00.22' performs emergency braking with decel=-63.72 wished=4.50 severity=13.16, time=121.30.
(pid=2086) Warning: Vehicle 'flow_10.171' performs emergency braking with decel=-10.81 wished=4.50 severity=1.40, time=124.20.
(pid=2086) Warning: Vehicle 'flow_00.21' performs emergency braking with decel=-12.94 wished=4.50 severity=1.88, time=124.30.
(pid=2086) Warning: Vehicle 'flow_00.22' performs emergency braking with decel=-58.25 wished=4.50 severity=11.95, 

...

(pid=2088) Warning: Vehicle 'flow_10.36' performs emergency braking with decel=-46.67 wished=4.50 severity=9.37, time=181.10.
(pid=2088) Warning: Vehicle 'flow_10.36' performs emergency braking with decel=-47.47 wished=4.50 severity=9.55, time=181.20.
(pid=2086) Warning: Vehicle 'flow_10.38' performs emergency braking with decel=-30.52 wished=4.50 severity=5.78, time=203.30.
(pid=2088) Warning: Vehicle 'flow_00.28' performs emergency braking with decel=-9.00 wished=4.50 severity=1.00, time=184.20.
(pid=2088) Warning: Vehicle 'flow_10.49' performs emergency braking with decel=-9.02 wished=4.50 severity=1.01, time=184.80.
(pid=2088) Warning: Vehicle 'flow_10.41' performs emergency braking with decel=-90.45 wished=4.50 severity=19.10, time=187.70.
(pid=2088) Warning: Vehicle 'flow_10.41' performs emergency braking with decel=-14.89 wished=4.50 severity=2.31, time=189.90.
(pid=2088) Warning: Vehicle 'flow_00.35' performs emergency braking with decel=-22.41 wished=4.50 severity=3.98, time=1

(pid=2088) Warning: Vehicle 'flow_10.1' performs emergency braking with decel=-83.47 wished=4.50 severity=17.55, time=130.70.
(pid=2088) Warning: Vehicle 'flow_00.24' performs emergency braking with decel=-23.54 wished=4.50 severity=4.23, time=130.80.
(pid=2088) Warning: Vehicle 'flow_10.1' performs emergency braking with decel=-11.84 wished=4.50 severity=1.63, time=132.80.
(pid=2086) Warning: Vehicle 'flow_00.26' performs emergency braking with decel=-9.00 wished=4.50 severity=1.00, time=156.10.
(pid=2086) Warning: Vehicle 'flow_00.18' performs emergency braking with decel=-117.65 wished=4.50 severity=25.14, time=156.40.
(pid=2086) Warning: Vehicle 'flow_10.10' performs emergency braking with decel=-14.42 wished=4.50 severity=2.20, time=156.50.
(pid=2088) Warning: Vehicle 'flow_00.14' performs emergency braking with decel=-39.47 wished=4.50 severity=7.77, time=134.30.
(pid=2088) Warning: Vehicle 'flow_00.14' performs emergency braking with decel=-52.02 wished=4.50 severity=10.56, time

(pid=2088) Warning: Vehicle 'flow_00.29' performs emergency braking with decel=-62.55 wished=4.50 severity=12.90, time=140.90.
(pid=2088) Warning: Vehicle 'flow_00.29' performs emergency braking with decel=-11.61 wished=4.50 severity=1.58, time=146.50.
(pid=2088) Warning: Vehicle 'flow_00.28' performs emergency braking with decel=-18.81 wished=4.50 severity=3.18, time=154.80.
(pid=2088) Warning: Vehicle 'flow_10.16' performs emergency braking with decel=-10.87 wished=4.50 severity=1.42, time=161.20.
(pid=2088) Warning: Vehicle 'flow_10.35' performs emergency braking with decel=-11.27 wished=4.50 severity=1.51, time=168.40.
(pid=2088) Warning: Vehicle 'flow_00.33' performs emergency braking with decel=-23.07 wished=4.50 severity=4.13, time=176.50.
(pid=2086) Warning: Vehicle 'flow_00.35' performs emergency braking with decel=-23.32 wished=4.50 severity=4.18, time=198.50.
(pid=2088) Warning: Vehicle 'flow_10.43' performs emergency braking with decel=-135.72 wished=4.50 severity=29.16, ti

(pid=2086) Warning: Vehicle 'flow_10.23' performs emergency braking with decel=-9.59 wished=4.50 severity=1.13, time=162.70.
(pid=2086) Warning: Vehicle 'flow_10.37' performs emergency braking with decel=-55.43 wished=4.50 severity=11.32, time=182.70.
(pid=2086) Warning: Vehicle 'flow_00.38' performs emergency braking with decel=-24.39 wished=4.50 severity=4.42, time=184.10.
(pid=2088) Warning: Vehicle 'flow_00.37' performs emergency braking with decel=-9.00 wished=4.50 severity=1.00, time=208.00.
(pid=2086) Warning: Vehicle 'flow_10.17' performs emergency braking with decel=-10.09 wished=4.50 severity=1.24, time=185.10.
(pid=2088) Warning: Vehicle 'flow_00.40' performs emergency braking with decel=-21.11 wished=4.50 severity=3.69, time=210.40.
(pid=2086) Warning: Vehicle 'flow_10.5' performs emergency braking with decel=-14.13 wished=4.50 severity=2.14, time=192.70.
(pid=2086) Warning: Vehicle 'flow_10.18' performs emergency braking with decel=-11.33 wished=4.50 severity=1.52, time=20

(pid=2088) Warning: Vehicle 'flow_10.27' performs emergency braking with decel=-50.12 wished=4.50 severity=10.14, time=164.10.
(pid=2086) Warning: Vehicle 'flow_00.24' performs emergency braking with decel=-25.59 wished=4.50 severity=4.69, time=136.30.
(pid=2088) Warning: Vehicle 'flow_00.25' performs emergency braking with decel=-9.00 wished=4.50 severity=1.00, time=168.90.
(pid=2086) Warning: Vehicle 'flow_00.29' performs emergency braking with decel=-10.62 wished=4.50 severity=1.36, time=149.10.
(pid=2088) Warning: Vehicle 'flow_00.47' performs emergency braking with decel=-9.00 wished=4.50 severity=1.00, time=181.50.
(pid=2086) Warning: Vehicle 'flow_00.30' performs emergency braking with decel=-9.77 wished=4.50 severity=1.17, time=154.90.
(pid=2086) Warning: Vehicle 'flow_10.218' performs emergency braking with decel=-9.19 wished=4.50 severity=1.04, time=160.60.
(pid=2086) Warning: Vehicle 'flow_10.34' performs emergency braking with decel=-111.36 wished=4.50 severity=23.75, time=